Well well well. You are here. At my first solo project created by myself from scratch. This will have some... playful aspects (hopefully) and I guess a little insight of my inner thoughts. No filters. This is me. Enjoy :-)

# The Great Beginning

## Somaan's Personal Expense Dashboard

This notebook houses all the code cells needed to load or create views of provided csv files for planned and actual spend.
If it works as expected, we shall get some colourful pie charts depicting the spends of each planned and actual spends with categories as well as a third showing the difference. That third one will need extra thought to think about whether a pie chart is the best way to show that. Maybe a two-way bar chart. All in the journey of a new data scientist!

### Setup
Run the code cell below to set up the notebook

In [2]:
import pandas as pd
import numpy as np

## Remaining Steps.

The rest of the steps will take place in the following order.

- Step 2: Load csv files

- Step 3: Analyze Your Spending

- Step 4: Visualize Spending

- Step 5: Optional Improvements

### Step 2: Load csv files

The csv files in the github repository are named 'planned_spend_march.csv' and 'actual_spend_march.csv'. Both of these files will be loaded. 
It is good practice to check both files once loaded to see if the column headers match and you can make sense of them.


In [3]:
planned_spend = pd.read_csv('planned_spend_march.csv')
actual_spend = pd.read_csv('actual_spend_march.csv')

# Display the first few rows of each DataFrame to check the structure and headers
print("Planned Spend DataFrame:")
print(planned_spend.head())

print("\nActual Spend DataFrame:")
print(actual_spend.head())

Planned Spend DataFrame:
      item_name  amount_GBP item_category       item_group
0          rent      1300.0  Fundamentals  Living Expenses
1           tfl        17.6  Fundamentals           Travel
2  Lunch @ work         8.8  Fundamentals  Living Expenses
3    Phone Bill        20.0  Fundamentals  Living Expenses
4      Coursera        38.0       Future         Education

Actual Spend DataFrame:
      item_name  amount_GBP item_category       item_group
0          rent      1300.0  Fundamentals  Living Expenses
1           tfl         0.0  Fundamentals           Travel
2  Lunch @ work         0.0  Fundamentals  Living Expenses
3    Phone Bill        20.0  Fundamentals  Living Expenses
4      Coursera        38.0       Future         Education


### Playground
Feel free to use the code cell below to play around with the dataframes. Do what you want. Go crazy!!!! ;-P

In [12]:
grouped_planned_spend_category = planned_spend.groupby('item_category')
grouped_planned_spend_category.describe()


amount_GBP                                                \
                   count        mean         std    min     25%    50%   
item_category                                                            
Fun                  3.0   22.990000   32.055578   3.98   4.485   4.99   
Fundamentals         7.0  251.628571  477.317790   8.80  16.300  20.00   
Future               5.0  103.044000  144.039056  25.00  37.220  38.00   
Miscellaneous        2.0   75.000000   35.355339  50.00  62.500  75.00   

                                
                   75%     max  
item_category                   
Fun             32.495    60.0  
Fundamentals   200.000  1300.0  
Future          55.000   360.0  
Miscellaneous   87.500   100.0

In [ ]:
grouped_actual_spend_category = actual_spend.groupby('item_category')
grouped_actual_spend_category.describe()

amount_GBP                                                  \
                   count        mean         std    min      25%     50%   
item_category                                                              
Fun                  4.0   43.907500   65.242827   3.98   4.7375  15.540   
Fundamentals         7.0  270.515714  487.995770   0.00   7.5000  20.000   
Future               5.0   68.322000   41.918235  25.00  38.0000  53.610   
Miscellaneous        2.0   53.065000    4.334565  50.00  51.5325  53.065   

                                  
                    75%      max  
item_category                     
Fun             54.7100   140.57  
Fundamentals   279.3050  1300.00  
Future         105.0000   120.00  
Miscellaneous   54.5975    56.13

### STEP 3: Analysis of Spending.

This section will reveal general statistics of the data and over time can reveal trends such as regular over spending in a certain category. 

In [13]:
grouped_planned_spend_item_group = planned_spend.groupby('item_group')
grouped_planned_spend_item_group.describe()


amount_GBP                                           \
                          count        mean         std     min      25%   
item_group                                                                 
Charity                     1.0  340.000000         NaN  340.00  340.000   
Credit Repayment            3.0   39.073333   15.085627   25.00   31.110   
Eating Out                  1.0   60.000000         NaN   60.00   60.000   
Education                   1.0   38.000000         NaN   38.00   38.000   
Entertainment               3.0  122.990000  205.257302    3.98    4.485   
Gifts                       1.0   50.000000         NaN   50.00   50.000   
Groceries + whatever        1.0  100.000000         NaN  100.00  100.000   
Living Expenses             4.0  335.950000  642.716327    8.80   13.450   
Travel                      2.0   38.800000   29.981328   17.60   28.200   

                                               
                         50%      75%     max  
item_group                                     
Charity               340.00  340.000   340.0  
Credit Repayment       37.22   46.110    55.0  
Eating Out             60.00   60.000    60.0  
Education              38.00   38.000    38.0  
Entertainment           4.99  182.495   360.0  
Gifts                  50.00   50.000    50.0  
Groceries + whatever  100.00  100.000   100.0  
Living Expenses        17.50  340.000  1300.0  
Travel                 38.80   49.400    60.0

In [14]:
grouped_actual_spend_item_group = actual_spend.groupby('item_group')
grouped_actual_spend_item_group.describe()

amount_GBP                                            \
                          count        mean         std     min       25%   
item_group                                                                  
Charity                     1.0  496.610000         NaN  496.61  496.6100   
Credit Repayment            3.0   61.203333   40.536947   25.00   39.3050   
Eating Out                  1.0  140.570000         NaN  140.57  140.5700   
Education                   1.0   38.000000         NaN   38.00   38.0000   
Entertainment               4.0   38.765000   55.107555    3.98    4.7375   
Gifts                       1.0   50.000000         NaN   50.00   50.0000   
Groceries + whatever        1.0   56.130000         NaN   56.13   56.1300   
Living Expenses             4.0  333.750000  644.222723    0.00   11.2500   
Travel                      2.0   31.000000   43.840620    0.00   15.5000   

                                                 
                         50%       75%      max  
item_group                                       
Charity               496.61  496.6100   496.61  
Credit Repayment       53.61   79.3050   105.00  
Eating Out            140.57  140.5700   140.57  
Education              38.00   38.0000    38.00  
Entertainment          15.54   49.5675   120.00  
Gifts                  50.00   50.0000    50.00  
Groceries + whatever   56.13   56.1300    56.13  
Living Expenses        17.50  340.0000  1300.00  
Travel                 31.00   46.5000    62.00